In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI 
client = OpenAI()

In [3]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role":"system","content":"You are a helpful assistant."
        },
        {
            "role":"user","content":"Tell me a short joke about AI."
        }
    ]
)
print("Assistant:"+ completion.choices[0].message.content)

Assistant:Why was the AI at the party so confident?

Because it had a lot of "neural networks"!


In [4]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role":"system","content":"You are a helpful assistant that responds in Markdown. Help me with my math homework!"},
        { "role":"user","content":[
            {"type":"text", "text":"What's the area of the triangle?"},
            {"type":"image_url", "image_url": {
                "url":"https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"
            }}
        ]}
    ],
    temperature=0,
)
print("Assistant:"+ response.choices[0].message.content)

Assistant:To find the area of the triangle, we can use Heron's formula. First, we need to find the semi-perimeter \( s \) of the triangle.

The sides of the triangle are 6, 5, and 9.

The semi-perimeter \( s \) is calculated as:
\[ s = \frac{a + b + c}{2} \]
where \( a = 6 \), \( b = 5 \), and \( c = 9 \).

\[ s = \frac{6 + 5 + 9}{2} = \frac{20}{2} = 10 \]

Now, we can use Heron's formula to find the area \( A \):
\[ A = \sqrt{s(s-a)(s-b)(s-c)} \]

Substitute the values:
\[ A = \sqrt{10(10-6)(10-5)(10-9)} \]
\[ A = \sqrt{10 \cdot 4 \cdot 5 \cdot 1} \]
\[ A = \sqrt{200} \]
\[ A = 10\sqrt{2} \]

So, the area of the triangle is \( 10\sqrt{2} \) square units.


In [5]:
VIDEO_PATH = "data/bison.mp4"

In [6]:
import cv2
import os
from moviepy.editor import VideoFileClip
import time
import base64
from IPython.display import Image

def process_video(video_path, seconds_per_frame=2):
    base64Frames = []
    base_video_path, _ = os.path.splitext(video_path)

    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame=0

    # Loop through the video and extract frames at specified sampling rate
    while curr_frame < total_frames - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        curr_frame += frames_to_skip
    video.release()

    # Extract audio from video
    audio_path = f"{base_video_path}.mp3"
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path, bitrate="32k")
    clip.audio.close()
    clip.close()

    print(f"Extracted {len(base64Frames)} frames")
    print(f"Extracted audio to {audio_path}")
    return base64Frames, audio_path

In [7]:
base64Frames, audio_path = process_video(VIDEO_PATH, seconds_per_frame=1)

MoviePy - Writing audio in data/bison.mp3


MoviePy - Done.
Extracted 21 frames
Extracted audio to data/bison.mp3


In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are generating a video summary. Please provide a summary of the video. Respond in Markdown."},
        {
            "role": "user", 
            "content": [
                "These are the frames from the video.",
                *map(lambda x: {"type": "image_url", 
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}},
                                base64Frames)
            ],
        }
    ],
    temperature=0,
)
print(response.choices[0].message.content)

The video frames depict a dramatic wildlife scene in a snowy landscape. Here's a summary of the events:

1. **Initial Encounter**: A group of wolves surrounds a bison, likely targeting it as prey.
2. **Attack Begins**: The wolves begin to attack the bison, trying to bring it down.
3. **Bison Defense**: Another bison attempts to intervene and protect the targeted bison.
4. **Pack Attack**: More wolves join the attack, intensifying their efforts to overpower the bison.
5. **Struggle**: The bison tries to fend off the wolves, but the pack's persistence is evident.
6. **Overpowering**: The wolves eventually overpower the bison, bringing it down.
7. **Feeding**: The wolves start feeding on the fallen bison, with some standing guard while others eat.

The sequence highlights the harsh realities of survival in the wild, showcasing the wolves' hunting strategy and the bison's struggle to defend itself.


In [9]:
transcription = client.audio.transcriptions.create(
    model="whisper-1", 
    file=open(audio_path,"rb"),
)

In [10]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are generating a transcript summary. Create a summary of the provided transcription. Respond in Markdown."},
        {"role": "user", "content": [
        {"type": "text", "text": f"The audio transcription is: {transcription.text}"}
        ],}
    ],
    temperature=0,
)
print(response.choices[0].message.content)

The transcription describes a scene where a herd, focused solely on escaping, inadvertently provides an advantage to the pursuing wolves.


In [11]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are generating a video summary. Please provide a summary of the video. Respond in Markdown."},
        {
            "role": "user", 
            "content": [
                "These are the frames from the video.",
                *map(lambda x: {"type": "image_url", 
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}},base64Frames),
                                {"type": "text", "text": f"The audio transcription is: {transcription.text}"}
            ],
        }
    ],
    temperature=0,
)
print(response.choices[0].message.content)

The video frames depict a dramatic wildlife scene in a snowy landscape. Here's a summary of the video:

1. **Initial Scene**: A group of wolves is seen surrounding a bison in the snow.
2. **Attack Begins**: The wolves begin to attack the bison, attempting to bring it down.
3. **Bison's Struggle**: The bison struggles against the wolves, trying to fend them off.
4. **Herd Intervention**: Other bison from the herd rush in to help, attempting to drive the wolves away.
5. **Continued Attack**: Despite the herd's intervention, the wolves persist in their attack.
6. **Bison Falls**: Eventually, the bison falls to the ground, overwhelmed by the wolves.
7. **Wolves Feed**: The wolves start feeding on the fallen bison.
8. **Herd Retreats**: The rest of the bison herd retreats, leaving the wolves to their meal.

The audio transcription indicates that the herd's primary focus was on escaping, which provided an opportunity for the wolves to successfully hunt the bison.
